In [180]:
import os
import random
import json
import glob
import pickle
import csv

In [202]:
root_dir_dict = {
    "continental" : "/nas/EOS/servers/storage03-srv2/dataset/axcar/stereo",
    "torc": {
                "wk1":  "/nas/EOS/dataset/torc/wk1/fixed_structure",
                "wk2":  "/nas/EOS/dataset/torc/wk2/fixed_structure",
                "wk3":  "/nas/EOS/dataset/torc/wk3/fixed_structure"
    }
}
json_dir = "/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses"
NUM_IMAGES_PER_SEQUENCE = 500

In [203]:
json_files = glob.glob(os.path.join(json_dir, '**', '*.json'), recursive=True)

In [204]:
json_files

['/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-200.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-199.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-204.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-195.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-194.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-201.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-198.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-205.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-196.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-192.json',
 '/nas/EOS/users/aman/data/RelativePose/groundtruth_json_poses/torc/wk2/CAPT-203.json',
 '/nas/EOS/users/aman/data/Relat

In [205]:
dataset = "torc"
week = None if dataset == "continental" else "wk2"
capture = "CAPT-199"

In [206]:
# Read the json file
if week is None:
    json_fpath = os.path.join(json_dir, dataset, f'{capture}.json')
    root = root_dir_dict[dataset]
else:
    json_fpath = os.path.join(json_dir, dataset , week, f'{capture}.json')
    root = root_dir_dict[dataset][week]

In [207]:
# Load json data
with open(json_fpath,'r') as fp:
    cam_data = json.load(fp)

# load which type of cameras are used in this dataset
cam_types = list(cam_data[capture].keys())
cam_nums = {cam_type : list(cam_data[capture][cam_type]['intrinsics'].keys()) for cam_type in cam_types}

# Load all possible image camera pairs in JSON file
cam_pairs = {cam_type : list(cam_data[capture][cam_type]['extrinsics'].keys()) for cam_type in cam_types}

In [208]:
cam_pairs

{'ar0820': ['0_1', '0_2', '0_3', '1_2', '1_3', '2_3']}

In [209]:
frames_data = []
for cam_type in cam_types:

    # possible image sequences containing camera name and image#
    seq_paths = glob.glob(f'{root}/{capture}/*/{cam_type}/image{cam_nums[cam_type][0]}')

    # filter based on few words
    words = ['night', 'lidar', 'calib']
    seq_paths = list(filter(lambda x : all([word not in x for word in words]), seq_paths))

    # get sequence names
    seq_names = list(map(lambda x : x.split('/')[-3], seq_paths))
    

    for seq in seq_names:
        for cam1, cam2 in map(lambda x : tuple(x.split('_')), list(cam_pairs.values())[0]):
            try:
                left_fnames = sorted(os.listdir(os.path.join(root, capture, seq, cam_type, 'image'+str(cam1))))
                right_fnames = sorted(os.listdir(os.path.join(root, capture, seq, cam_type, 'image'+str(cam2))))

                if len(left_fnames) != len(right_fnames):
                    raise Exception
                else:
                    num_images = len(left_fnames)
                    sample_size = NUM_IMAGES_PER_SEQUENCE//len(seq_names) + 1
                    sample_indxs = random.sample(range(0, num_images), sample_size)

                    for sample_indx in sample_indxs:
                        frames_data.append([root, capture, seq, cam_type,
                                            f"image{cam1}/{left_fnames[sample_indx].split('.')[0]}",
                                            f"image{cam2}/{left_fnames[sample_indx].split('.')[0]}"])

            except:
                print(f"Number of left({len(left_fnames)}) and right images({len(right_fnames)}) not same!, Skipping  sequence : {seq} image pair : {cam1}, {cam2}")
                



In [210]:
frames_data

[['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
  'image0/frame_image0_00012336',
  'image1/frame_image0_00012336'],
 ['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
  'image0/frame_image0_00017144',
  'image1/frame_image0_00017144'],
 ['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
  'image0/frame_image0_00017384',
  'image1/frame_image0_00017384'],
 ['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
  'image0/frame_image0_00007287',
  'image1/frame_image0_00007287'],
 ['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
  'image0/frame_image0_00013829',
  'image1/frame_image0_00013829'],
 ['/nas/EOS/dataset/torc/wk2/fixed_structure',
  'CAPT-199',
  'dusk_3_CAPT-199_20211117192618',
  'ar0820',
 

In [211]:
left_fnames = sorted(os.listdir(os.path.join(root, capture, seq, cam_type, 'image'+str(cam1))))

In [212]:
right_fnames = sorted(os.listdir(os.path.join(root, capture, seq, cam_type, 'image'+str(cam2))))

In [213]:
left_fnames

['frame_image2_00000907.raw',
 'frame_image2_00000908.raw',
 'frame_image2_00000909.raw',
 'frame_image2_00000910.raw',
 'frame_image2_00000911.raw',
 'frame_image2_00000912.raw',
 'frame_image2_00000913.raw',
 'frame_image2_00000914.raw',
 'frame_image2_00000915.raw',
 'frame_image2_00000916.raw',
 'frame_image2_00000917.raw',
 'frame_image2_00000918.raw',
 'frame_image2_00000919.raw',
 'frame_image2_00000920.raw',
 'frame_image2_00000921.raw',
 'frame_image2_00000922.raw',
 'frame_image2_00000923.raw',
 'frame_image2_00000924.raw',
 'frame_image2_00000925.raw',
 'frame_image2_00000926.raw',
 'frame_image2_00000927.raw',
 'frame_image2_00000928.raw',
 'frame_image2_00000929.raw',
 'frame_image2_00000930.raw',
 'frame_image2_00000931.raw',
 'frame_image2_00000932.raw',
 'frame_image2_00000933.raw',
 'frame_image2_00000934.raw',
 'frame_image2_00000935.raw',
 'frame_image2_00000936.raw',
 'frame_image2_00000937.raw',
 'frame_image2_00000938.raw',
 'frame_image2_00000939.raw',
 'frame_im

In [214]:
right_fnames

['frame_image3_00000907.raw',
 'frame_image3_00000908.raw',
 'frame_image3_00000909.raw',
 'frame_image3_00000910.raw',
 'frame_image3_00000911.raw',
 'frame_image3_00000912.raw',
 'frame_image3_00000913.raw',
 'frame_image3_00000914.raw',
 'frame_image3_00000915.raw',
 'frame_image3_00000916.raw',
 'frame_image3_00000917.raw',
 'frame_image3_00000918.raw',
 'frame_image3_00000919.raw',
 'frame_image3_00000920.raw',
 'frame_image3_00000921.raw',
 'frame_image3_00000922.raw',
 'frame_image3_00000923.raw',
 'frame_image3_00000924.raw',
 'frame_image3_00000925.raw',
 'frame_image3_00000926.raw',
 'frame_image3_00000927.raw',
 'frame_image3_00000928.raw',
 'frame_image3_00000929.raw',
 'frame_image3_00000930.raw',
 'frame_image3_00000931.raw',
 'frame_image3_00000932.raw',
 'frame_image3_00000933.raw',
 'frame_image3_00000934.raw',
 'frame_image3_00000935.raw',
 'frame_image3_00000936.raw',
 'frame_image3_00000937.raw',
 'frame_image3_00000938.raw',
 'frame_image3_00000939.raw',
 'frame_im

In [133]:
list(map(lambda x : tuple(x.split('_')), list(cam_pairs.values())[0]))

[('0', '1'), ('0', '2'), ('0', '3'), ('1', '2'), ('1', '3'), ('2', '3')]

In [111]:
f'{root}/{capture}/*/{cam_type}/image{cam_nums[cam_type][0]}'

'/nas/EOS/dataset/torc/wk3/fixed_structure/CAPT-356/*/ar0820/image0'

In [112]:
seq_names

['noon_CAPT-356_20211208133513']

In [37]:
w

False

In [38]:
seq_paths[10]

'/nas/EOS/servers/storage03-srv2/dataset/axcar/stereo/CAR-502/night_loop_2_2021-06-02__01h-53m-55s/fsc231/image0'